# PIZZA OpenAI Examples

Various examples working with the `OpenAIAttributor`. 

Before using this notebook, make sure to set your `OPENAI_API_KEY` evironment variable in a `.env` file.

## Setup

Setting up the attributor and consistent input arguments

In [7]:
import warnings

from attribution.api_attribution import OpenAIAttributor
from attribution.experiment_logger import ExperimentLogger
from attribution.token_perturbation import (
    FixedPerturbationStrategy,
    NthNearestPerturbationStrategy,
    calculate_chunk_size,
    get_units_from_prompt,
)

warnings.simplefilter(action='ignore', category=FutureWarning)

# Re-import modified modules without restarting the server
%load_ext autoreload
%autoreload 2

# Load environment variables (OpenAI API key)
%load_ext dotenv
%dotenv

attributor = OpenAIAttributor(request_chunksize=10)
perturbation_strategy = FixedPerturbationStrategy("")

kwargs = {
    "attribution_strategies": ["cosine"],  
    "perturb_word_wise": True,
    "ignore_output_token_location": True,
}

## Short prompts
Comparing full perturbation and hierarchical perturbation methods

In [8]:
mixed_prompts = [
    "What is the capital of the country that was formerly known as Siam? Answer in 1 word.",
    "What is the chemical element with the symbol 'Au'? Answer in 1 word.",
    "Which character in 'Pride and Prejudice' said 'It is a truth universally acknowledged'? Answer in 1 word.",
    "Who holds the record for the most goals in a calendar year in football (soccer)? Answer in 1 word.",
    "In Greek mythology, who is the goddess of wisdom and warfare? Answer in 1 word.",
    "Who directed the film 'Inception'? Answer in 1 word.",
    "Who is known as the 'King of Pop'? Answer in 1 word.",
    "Who was the first female Prime Minister of the United Kingdom? Answer in 1 word.",
    "Who is the co-founder of Microsoft? Answer in 1 word.",
]

for input_str in mixed_prompts:
    response = await attributor.get_chat_completion(input_str)

    logger = ExperimentLogger()

    await attributor.compute_attributions(
        input_str,
        perturbation_strategy=perturbation_strategy,
        logger=logger,
        **kwargs
    )
    
    saliency = await attributor.hierarchical_perturbation(
        input_str,
        init_chunk_size=4,
        perturbation_strategy=perturbation_strategy,
        logger=logger,
        verbose=True,
        **kwargs
    )

    logger.print_sentence_attribution()
    display(logger.df_experiments)

Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]


Stage 0: making 5 perturbations
Masked out tokens/words:
['What is']
['the capital of the']
['country that was formerly']
['known as Siam? Answer']
['in 1 word.']
Stage 1: making 6 perturbations
Masked out tokens/words:
['1 word.']
['in']
['Siam? Answer']
['known as']
['of the']
['the capital']
Stage 2: making 6 perturbations
Masked out tokens/words:
['Answer']
['Siam?']
['the']
['of']
['capital']
['the']


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17
0,1,cosine,fixed,True,What 0.00,is 0.00,the 0.00,capital 0.59,of 0.00,the 0.00,country 0.00,that 0.00,was 0.00,formerly 0.00,known 0.00,as 0.00,Siam? 0.66,Answer 0.00,in 0.00,1 0.00,word. 0.00
1,2,cosine,fixed,True,What 0.00,is 0.00,the 0.00,capital 0.59,of 0.00,the 0.00,country 0.00,that 0.00,was 0.00,formerly 0.00,known 0.00,as 0.00,Siam? 0.66,Answer 0.00,in 0.00,1 0.00,word. 0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,What is the capital of the country that was fo...,Bangkok,fixed,True,2.189686,18
1,2,What is the capital of the country that was fo...,Bangkok,fixed,True,2.839399,18


Sending 10 concurrent requests at a time: 100%|██████████| 2/2 [00:01<00:00,  1.90it/s]


...

,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15
0,1,cosine,fixed,True,Who -0.00,was -0.00,the -0.00,first 0.69,female 0.68,Prime -0.00,Minister -0.00,of -0.00,the -0.00,United 0.68,Kingdom? -0.00,Answer 0.69,in -0.00,1 -0.00,word. -0.00
1,2,cosine,fixed,True,Who -0.00,was -0.00,the -0.00,first 0.69,female 0.68,Prime -0.00,Minister -0.00,of -0.00,the -0.00,United 0.68,Kingdom? 0.12,Answer 0.12,in 0.12,1 0.12,word. -0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who was the first female Prime Minister of the...,Margaret,fixed,True,1.135532,16
1,2,Who was the first female Prime Minister of the...,Margaret,fixed,True,2.7139,18


Stage 0: making 3 perturbations
Masked out tokens/words:
['Who is']
['the co-founder of Microsoft?']
['Answer in 1 word.']
Stage 1: making 2 perturbations
Masked out tokens/words:
['of Microsoft?']
['the co-founder']
Stage 2: making 2 perturbations
Masked out tokens/words:
['Microsoft?']
['of']


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10
0,1,cosine,fixed,True,Who 0.00,is 0.54,the 0.00,co-founder 0.64,of 0.00,Microsoft? 0.51,Answer 0.00,in 0.00,1 -0.00,word. -0.00
1,2,cosine,fixed,True,Who 0.00,is 0.00,the 0.00,co-founder 0.00,of 0.00,Microsoft? 0.51,Answer 0.04,in 0.04,1 0.04,word. 0.04


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who is the co-founder of Microsoft? Answer in ...,Bill,fixed,True,0.768504,11
1,2,Who is the co-founder of Microsoft? Answer in ...,Bill,fixed,True,1.823514,8


## Longer prompts
Using hierarchical perturbation only

In [9]:
longer_prompts = [
    "In J.K. Rowling's Harry Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",
    "In the field of astronomy, there is a phenomenon where the light from a star is bent and magnified by the gravitational field of another object, such as a galaxy or black hole, that lies between the star and the observer. This effect was first predicted by Einstein's theory of general relativity. What is this phenomenon called? Answer in 1 word.",
    "Located in South America, there is a vast river that flows through Brazil, Peru, and several other countries. It is the largest river by discharge volume of water in the world and is often associated with the rainforest of the same name. What is the name of this river? Answer in 1 word.",
    "In Norse mythology, there is a hammer wielded by the god Thor, which is renowned for its immense power and is said to be capable of leveling mountains. This hammer is also a symbol of protection and blessing. What is the name of Thor's hammer? Answer in 1 word.",
    "In the movie 'The Matrix,' the protagonist is a computer hacker who learns about the true nature of his reality and his role in the war against its controllers. He is given a choice between two pills: a red pill that reveals the truth, and a blue pill that returns him to his normal life. What is the name of the protagonist? Answer in 1 word.",
    "There is a famous painting by Vincent van Gogh that depicts a night sky filled with swirling clouds, stars, and a bright crescent moon. This painting is one of his most well-known works and was created while he was in a mental asylum in Saint-Rémy-de-Provence. What is the title of this painting? Answer in 1 word.",
    "In classical music, there is a composer who is renowned for his symphonies, concertos, and sonatas. Born in Salzburg in 1756, he began composing music at a very young age and created over 600 works during his lifetime. What is the last name of this composer? Answer in 1 word.",
    "In the realm of computer programming, there is a widely used language that was developed by Guido van Rossum and first released in 1991. It emphasizes code readability and its syntax allows programmers to express concepts in fewer lines of code. What is the name of this programming language? Answer in 1 word.",
    "In the study of genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",
]

for input_str in longer_prompts:

    logger = ExperimentLogger()

    await attributor.hierarchical_perturbation(
        input_str,
        init_chunk_size=8,
        perturbation_strategy=perturbation_strategy,
        logger=logger,
        **kwargs
    )

    logger.print_sentence_attribution()
    display(logger.df_experiments)
    logger.print_attribution_matrix(exp_id=1, show_debug_cols=True)

Stage 0: making 8 perturbations
Stage 1: making 4 perturbations
Stage 2: making 4 perturbations
Stage 3: making 2 perturbations


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55,token_56
0,1,cosine,fixed,True,In -0.00,J.K. -0.00,Rowling's -0.00,Harry -0.00,Potter -0.00,"series, -0.00",the -0.00,spell -0.00,used -0.00,to -0.00,conjure -0.00,a -0.00,Patronus -0.00,is -0.00,considered -0.00,highly -0.00,advanced -0.00,and -0.00,can -0.00,only -0.00,be -0.00,performed -0.00,by -0.00,skilled -0.00,witches -0.00,and -0.00,wizards. -0.00,The -0.00,form -0.00,that -0.00,a -0.00,Patronus -0.00,takes -0.00,can -0.00,vary -0.00,widely -0.00,and -0.00,is -0.00,often -0.00,influenced -0.00,by -0.00,the -0.00,caster's -0.00,personality -0.00,and -0.00,experiences. -0.00,What -0.00,form -0.00,does -0.00,Snape's 0.66,Patronus -0.00,take? -0.00,Answer -0.00,in -0.00,1 -0.00,word. 0.56


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In J.K. Rowling's Harry Potter series, the spe...",Doe,fixed,True,4.912997,19


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,D (0),oe (1),perturbed_input,perturbed_output
In (0),0.000000,-0.000000,"Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",Doe
J.K. (1),0.000000,-0.000000,"Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",Doe
Rowling's (2),0.000000,-0.000000,"Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",Doe
Harry (3),0.000000,-0.000000,"Potter series, the spell used to conjure a Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.",Doe
Potter (4),0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe
"series, (5)",0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe
the (6),0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe
spell (7),0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe
used (8),0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe
to (9),0.000000,-0.000000,In J.K. Rowling's Harry Patronus is considered highly advanced and can only be performed by skilled witches and wizards. The form that a Patronus takes can vary widely and is often influenced by the caster's personality and experiences. What form does Snape's Patronus take? Answer in 1 word.,Doe


...

Stage 0: making 8 perturbations
Stage 1: making 2 perturbations
Stage 2: making 2 perturbations
Stage 3: making 4 perturbations


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55,token_56,token_57,token_58,token_59
0,1,cosine,fixed,True,In -0.00,the -0.00,study -0.00,of -0.00,"genetics, -0.00",there -0.00,is -0.00,a -0.00,molecule -0.00,that -0.00,carries -0.00,the -0.00,genetic -0.00,instructions -0.00,used -0.00,in -0.00,the -0.00,"growth, -0.00","development, -0.00","functioning, -0.00",and -0.00,reproduction -0.00,of -0.00,all -0.00,known -0.00,living -0.00,organisms -0.00,and -0.00,many -0.00,viruses. -0.00,This -0.00,molecule -0.00,is -0.00,structured -0.00,as -0.00,a -0.00,double -0.00,helix -0.00,and -0.00,was -0.00,first -0.00,described -0.00,by -0.00,Watson -0.00,and -0.00,Crick -0.00,in -0.00,1953. -0.00,What -0.00,is -0.00,the -0.00,abbreviation -0.00,for -0.00,this -0.00,molecule? -0.00,Answer -0.00,in -0.00,1 0.29,word. -0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"In the study of genetics, there is a molecule ...",DNA,fixed,True,4.476589,17


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,DNA (0),perturbed_input,perturbed_output
In (0),-0.000000,"genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
the (1),-0.000000,"genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
study (2),-0.000000,"genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
of (3),-0.000000,"genetics, there is a molecule that carries the genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
"genetics, (4)",-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
there (5),-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
is (6),-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
a (7),-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
molecule (8),-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA
that (9),-0.000000,"In the study of genetic instructions used in the growth, development, functioning, and reproduction of all known living organisms and many viruses. This molecule is structured as a double helix and was first described by Watson and Crick in 1953. What is the abbreviation for this molecule? Answer in 1 word.",DNA


## Calculating chunksize
Instead of specifying an initial chunksize directly, one can be compute based on a fraction or number of desired windows

In [10]:
mixed_prompts = [
    "Who painted the 'Mona Lisa'? Answer in 1 word.",
    "During the American Civil War, there was a significant battle fought from July 1 to July 3, 1863, which is often considered the turning point of the war. This battle took place in Pennsylvania and ended with a decisive victory for the Union forces. What is the name of this battle? Answer in 1 word.",
]

for input_str in mixed_prompts:
    response = await attributor.get_chat_completion(input_str)

    logger = ExperimentLogger()

    units, _, _ = get_units_from_prompt(input_str, attributor.tokenizer, perturb_word_wise=True)
    chunksize = calculate_chunk_size(len(units), fraction=0.25)  
    print(f"Using chunksize: {chunksize}")

    saliency = await attributor.hierarchical_perturbation(
        input_str,
        init_chunk_size=chunksize,
        perturbation_strategy=perturbation_strategy,
        logger=logger,
        **kwargs
    )

    logger.print_sentence_attribution()
    display(logger.df_experiments)
    logger.print_attribution_matrix(exp_id=1)

Using chunksize: 2
Stage 0: making 5 perturbations
Stage 1: making 2 perturbations


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9
0,1,cosine,fixed,True,Who 0.00,painted 0.75,the 0.00,'Mona 0.00,Lisa'? 0.00,Answer 0.10,in 0.10,1 0.07,word. 0.07


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,Who painted the 'Mona Lisa'? Answer in 1 word.,Da Vinci,fixed,True,1.494245,8


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,Da (0),Vin (1),ci (2)
Who (0),-0.000000,0.000000,0.000000
painted (1),0.708995,0.766966,0.780184
the (2),-0.000000,0.000000,0.000000
'Mona (3),-0.000000,0.000000,0.000000
Lisa'? (4),-0.000000,0.000000,0.000000
Answer (5),-0.000000,0.299857,0.000000
in (6),-0.000000,0.299857,0.000000
1 (7),0.215313,0.000000,0.000000
word. (8),0.215313,0.000000,0.000000


Using chunksize: 13
Stage 0: making 5 perturbations
Stage 1: making 2 perturbations
Stage 2: making 2 perturbations
Stage 3: making 2 perturbations
Stage 4: making 2 perturbations


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24,token_25,token_26,token_27,token_28,token_29,token_30,token_31,token_32,token_33,token_34,token_35,token_36,token_37,token_38,token_39,token_40,token_41,token_42,token_43,token_44,token_45,token_46,token_47,token_48,token_49,token_50,token_51,token_52,token_53,token_54,token_55
0,1,cosine,fixed,True,During 0.00,the 0.00,American 0.00,Civil 0.00,"War, 0.00",there 0.00,was 0.00,a 0.00,significant 0.00,battle 0.00,fought 0.00,from 0.00,July 0.00,1 0.00,to 0.00,July 0.00,"3, 0.00","1863, 0.00",which 0.00,is 0.00,often 0.00,considered 0.00,the 0.00,turning 0.00,point 0.00,of 0.00,the 0.00,war. 0.00,This 0.00,battle 0.00,took 0.00,place 0.00,in 0.00,Pennsylvania 0.00,and 0.00,ended 0.00,with 0.00,a 0.00,decisive 0.00,victory 0.00,for 0.00,the 0.00,Union 0.00,forces. 0.00,What 0.00,is 0.00,the 0.00,name 0.00,of 0.00,this 0.00,battle? 0.00,Answer 0.00,in 0.00,1 0.00,word. 0.00


,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,"During the American Civil War, there was a sig...",Gettysburg,fixed,True,4.68226,14


Attribution matrix for experiment 1 
Attribution Strategy: cosine 
Perturbation strategy: fixed:
Input Tokens (Rows) vs. Output Tokens (Columns)


,Getty (0),sburg (1)
During (0),0.000000,0.000000
the (1),0.000000,0.000000
American (2),0.000000,0.000000
Civil (3),0.000000,0.000000
"War, (4)",0.000000,0.000000
there (5),0.000000,0.000000
was (6),0.000000,0.000000
a (7),0.000000,0.000000
significant (8),0.000000,0.000000
battle (9),0.000000,0.000000


## Answer only in the question prompts
Using different perturbation strategies

In [11]:
perturbation_strategies = [
    FixedPerturbationStrategy(),
    NthNearestPerturbationStrategy(n=0),
    NthNearestPerturbationStrategy(n=-1),
]

In [12]:
self_fulfilling_prompts = [
    "The clock shows 9:47 PM. How many minutes to 10PM?",
    "Maria is 37 years old today. How many years till she's 50?",
    "John has 83 books on his shelf. If he buys 17 more books, how many books will he have in total?",
    "The building is 132 meters tall. How many centimeters tall is the building? No explanation",
    "The package weighs 8.6 kilograms. How many grams does the package weigh?",
    "Jack has 12 teaspoons of sugar. How many tablespoons of sugar does he have?",
    "Alex saved $363 from his birthday gifts. If he spends $45 on a new game, how much money will he have left? No explanation.",
    "The building is 132 meters tall. How many centimeters tall is the building? No explanation.",
    "The thermometer reads 23 degrees Celsius. What is the temperature in Fahrenheit? No explanation.",
    "There are 12 eggs in a dozen. If you use 5 eggs, how many eggs are left?",
]

# Initialize the logger
logger = ExperimentLogger()

# Perform the experiment
for input_str in self_fulfilling_prompts:
    print(f"Running prompt: '{input_str}'")
    
    for perturbation_strategy in perturbation_strategies:
        print(f"Using perturbation strategy: {perturbation_strategy}")
        await attributor.hierarchical_perturbation(
            input_str,
            init_chunk_size=4,
            perturbation_strategy=perturbation_strategy,
            logger=logger,
            **kwargs
        )

# Display the results
display(logger.df_experiments)
logger.print_sentence_attribution()

Running prompt: 'The clock shows 9:47 PM. How many minutes to 10PM?'
Using perturbation strategy: fixed
Stage 0: making 3 perturbations
Stage 1: making 2 perturbations
Using perturbation strategy: nth_nearest (n=0)
Stage 0: making 3 perturbations
Stage 1: making 2 perturbations
Using perturbation strategy: nth_nearest (n=-1)
Stage 0: making 3 perturbations
Stage 1: making 2 perturbations
Stage 2: making 4 perturbations
Running prompt: 'Maria is 37 years old today. How many years till she's 50?'
Using perturbation strategy: fixed
Stage 0: making 4 perturbations
Stage 1: making 2 perturbations
Using perturbation strategy: nth_nearest (n=0)
Stage 0: making 4 perturbations
Stage 1: making 4 perturbations
Stage 2: making 2 perturbations
Using perturbation strategy: nth_nearest (n=-1)
Stage 0: making 4 perturbations
Stage 1: making 4 perturbations
Running prompt: 'John has 83 books on his shelf. If he buys 17 more books, how many books will he have in total?'
Using perturbation strategy: fix

,exp_id,original_input,original_output,perturbation_strategy,perturb_word_wise,duration,num_llm_calls
0,1,The clock shows 9:47 PM. How many minutes to 1...,There are 13 minutes left until 10:00 PM.,fixed,True,2.580821,6
1,2,The clock shows 9:47 PM. How many minutes to 1...,13 minutes.,nth_nearest (n=0),True,4.651964,6
2,3,The clock shows 9:47 PM. How many minutes to 1...,There are 13 minutes left until 10:00 PM.,nth_nearest (n=-1),True,7.449116,10
3,4,Maria is 37 years old today. How many years ti...,Maria is 13 years away from turning 50.,fixed,True,2.94316,7
4,5,Maria is 37 years old today. How many years ti...,Maria is 13 years away from turning 50.,nth_nearest (n=0),True,7.732944,11
5,6,Maria is 37 years old today. How many years ti...,Maria is 13 years away from turning 50.,nth_nearest (n=-1),True,5.861634,9
6,7,John has 83 books on his shelf. If he buys 17 ...,John will have 100 books in total. \n\n83 + 17...,fixed,True,5.529009,19
7,8,John has 83 books on his shelf. If he buys 17 ...,John will have 100 books in total. \n\n83 + 17...,nth_nearest (n=0),True,13.495154,27
8,9,John has 83 books on his shelf. If he buys 17 ...,John will have 83 + 17 = 100 books in total.,nth_nearest (n=-1),True,9.49277,15
9,10,The building is 132 meters tall. How many cent...,"13,200 centimeters",fixed,True,3.717944,16


,exp_id,attribution_strategy,perturbation_strategy,perturb_word_wise,token_1,token_2,token_3,token_4,token_5,token_6,token_7,token_8,token_9,token_10,token_11,token_12,token_13,token_14,token_15,token_16,token_17,token_18,token_19,token_20,token_21,token_22,token_23,token_24
0,1,cosine,fixed,True,The 0.57,clock 0.06,shows 0.05,9:47 0.05,PM. 0.05,How 0.05,many 0.01,minutes 0.01,to 0.01,10PM? 0.01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2,cosine,nth_nearest (n=0),True,The 0.00,clock 0.10,shows 0.00,9:47 0.00,PM. 0.00,How 0.00,many 0.00,minutes 0.00,to 0.00,10PM? 0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,3,cosine,nth_nearest (n=-1),True,The 0.06,clock 0.06,shows 0.02,9:47 0.02,PM. 0.02,How 0.02,many 0.00,minutes 0.57,to 0.57,10PM? 0.45,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,4,cosine,fixed,True,Maria 0.57,is 0.12,37 0.06,years 0.06,old 0.06,today. 0.06,How 0.08,many 0.08,years 0.08,till 0.08,she's 0.13,50? 0.13,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,5,cosine,nth_nearest (n=0),True,Maria 0.00,is 0.00,37 0.00,years 0.00,old 0.00,today. 0.00,How 0.00,many 0.00,years 0.00,till 0.00,she's 0.00,50? 0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,6,cosine,nth_nearest (n=-1),True,Maria 0.57,is 0.12,37 0.05,years 0.05,old 0.05,today. 0.05,How 0.08,many 0.08,years 0.08,till 0.08,she's 0.19,50? 0.27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,7,cosine,fixed,True,John 0.14,has 0.00,83 0.28,books 0.10,on 0.04,his 0.04,shelf. 0.00,If 0.00,he 0.00,buys 0.00,17 0.23,more 0.10,"books, 0.00",how 0.07,many 0.00,books 0.00,will 0.00,he 0.00,have 0.02,in 0.02,total? 0.02,nan,nan,nan
7,8,cosine,nth_nearest (n=0),True,John 0.00,has 0.00,83 0.07,books 0.07,on 0.00,his 0.00,shelf. 0.00,If 0.00,he 0.00,buys 0.07,17 0.00,more 0.07,"books, 0.00",how 0.07,many 0.00,books 0.00,will 0.07,he 0.00,have 0.00,in 0.00,total? 0.00,nan,nan,nan
8,9,cosine,nth_nearest (n=-1),True,John 0.02,has 0.02,83 0.02,books 0.02,on 0.02,his 0.02,shelf. 0.01,If 0.01,he 0.01,buys 0.01,17 0.12,more 0.03,"books, 0.01",how 0.01,many 0.01,books 0.01,will 0.01,he 0.01,have 0.03,in 0.03,total? 0.28,nan,nan,nan
9,10,cosine,fixed,True,The -0.00,building -0.00,is -0.00,132 0.34,meters 0.27,tall. -0.00,How -0.00,many -0.00,centimeters 0.53,tall -0.00,is 0.02,the 0.02,building? 0.02,No 0.02,explanation -0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan
